In [27]:
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
import gspread
# 環境設置
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys  # 鍵盤操作
from selenium.common.exceptions import NoSuchElementException, TimeoutException
# ---------- 等待頁面加載完成
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
#-----------------------
import pandas as pd
import pandas.io.formats.excel
import requests
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()
from urllib.request import HTTPError
from bs4 import BeautifulSoup
import time
'''
# 載入自定義函式
import import_ipynb
from crawlers import crawlers
'''

scope = ['https://www.googleapis.com/auth/spreadsheets']
creds = Credentials.from_service_account_file("C:\\Users\mayda\Downloads\\books-319701-17701ae5510b.json", scopes=scope)
gs = gspread.authorize(creds)
sheet = gs.open_by_url('https://docs.google.com/spreadsheets/d/17fJuHSGHnjHbyKJzTgzKpp1pe2J6sirK5QVjg2-8fFo/edit#gid=0')
worksheet = sheet.get_worksheet(0)

ISBN = "9789573317241"  # "哈利波特：神秘的魔法石"
# 設定 driver
my_options = Options()
my_options.add_argument("--incognito")  # 開啟無痕模式
# my_options.add_argument("--headless")  # 不開啟實體瀏覽器
capabilities = DesiredCapabilities.CHROME
capabilities["pageLoadStrategy"] = "none"

# 國立臺灣海洋大學、國立臺灣師範大學、中央研究院、國立陽明大學、國立中央大學、中原大學
# 國立臺灣科技大學、中國文化大學、輔仁大學
def easy_crawler(position, org, url, ISBN):
    try:
        # 組合成書的網址
        url = url + ISBN
        # 載入 html，如果發生 HTTPError，那麼就使用 requests.get(url, verify=False)
        try:
            dfs = pd.read_html(url, encoding="utf-8")
        except HTTPError:
            resp = requests.get(url, verify=False)  # 設定 verify=False，以解決 SSLError
            dfs = pd.read_html(resp.text, encoding="utf-8")
        # 定位表格
        tgt = dfs[position]
        
        tgt.insert(0, "連結", [url for i in range(tgt.shape[0])])
        tgt.insert(0, "圖書館", [org for i in range(tgt.shape[0])])
        return tgt
    except:
        print(f"「{url}」無法爬取！")

def klccab_crawler(org, ISBN, driver, wait):
    try:
        url = f"https://webpac.klccab.gov.tw/webpac/search.cfm?m=as&k0={ISBN}&t0=i&c0=and&y10=&y20=&cat0=&dt0=&l0=&lv0=&lc0="
        driver.get(url)

        time.sleep(5)  # 基隆的系統太詭異了，強制等待
        soup = BeautifulSoup(driver.page_source, "html.parser")
        results = len(soup.find_all("div", "list_box"))
        if results < 2:
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.list.list_border")))
            tgt = pd.read_html(driver.page_source)[0]
        else:
            tgt = []
            for li in soup.find_all("div", "list_box"):
                url_temp = "https://webpac.klccab.gov.tw/webpac/" + li.find("a", "btn")["href"]
                driver.get(url_temp)
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.list.list_border")))
                tgt.append(pd.read_html(driver.page_source, encoding="utf-8")[0])
            tgt = pd.concat(tgt, axis=0, ignore_index=True)
            tgt.insert(0, "連結", [url for i in range(tgt.shape[0])])
            tgt.insert(0, "圖書館", [org for i in range(tgt.shape[0])])
        return tgt
    except:
        print(f"「{url}」無法爬取！")

def organize_columns(df):    
    # 合併欄位
    # 處理 column 2：館藏地
    c2 = ["館藏地/室", "館藏室", "館藏地/館藏室", "館藏地", "典藏館"]
    df["c2"] = ""
    for c in c2:
        try:
            df["c2"] += df[c]
        except:
            pass
    
    # 處理 column 3：索書號
    c3 = ["索書號", "索書號/期刊合訂本卷期", "索書號 / 部冊號"]
    df["c3"] = ""
    for c in c3:
        try:
            df["c3"] += df[c]
        except:
            pass

    # 處理 column 4：館藏狀態
    c4 = ["狀態/到期日", "目前狀態 / 到期日", "館藏狀態", "處理狀態", '狀態 (說明)', '館藏現況 說明']
    df["c4"] = ""
    for c in c4:
        try:
            df["c4"] += df[c]
        except:
            pass
    
    # 解決編碼問題
    df["c2"] = df["c2"].str.replace("栅", "柵").str.replace("斈", "學")

    c1 = ["基隆市立公共圖書館", "國立臺灣海洋大學"]
    df["c1"] = ""
    c5 = [""]
    df["c5"] = ""

    # 直接生成另一個 DataFrame
    test = pd.DataFrame()
    test["圖書館"] = df["c1"]
    test["館藏地"] = df["c2"]
    test["索書號"] = df["c3"]
    test["館藏狀態"] = df["c4"]
    test["連結"] = df["c5"]
    
    return test

def Keelung(ISBN):
    output = []
    
    driver = webdriver.Chrome("C:\\Users\mayda\Downloads\chromedriver", options=my_options, desired_capabilities=capabilities)
    wait = WebDriverWait(driver, 10)
    
    output.append(
        klccab_crawler(
            "基隆市公共圖書館", ISBN,  driver, wait
        )
    )
    
    output.append(
        easy_crawler(
            2, "國立臺灣海洋大學", "https://ocean.ntou.edu.tw/search*cht/i?SEARCH=", ISBN
        )
    )
    
    driver.quit()
    return organize_columns(pd.concat(output, axis=0, ignore_index=True).fillna(""))

Keelung(ISBN)
aa = Keelung(ISBN)
worksheet.update([aa.columns.values.tolist()] + aa.values.tolist())

{'spreadsheetId': '17fJuHSGHnjHbyKJzTgzKpp1pe2J6sirK5QVjg2-8fFo',
 'updatedRange': "'工作表1'!A1:E14",
 'updatedRows': 14,
 'updatedColumns': 5,
 'updatedCells': 70}

In [29]:
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
import gspread
# 環境設置
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys  # 鍵盤操作
from selenium.common.exceptions import NoSuchElementException, TimeoutException
# ---------- 等待頁面加載完成
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
#-----------------------
import pandas as pd
import pandas.io.formats.excel
import requests
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()
from urllib.request import HTTPError
from bs4 import BeautifulSoup
import time

# 載入自定義函式
import import_ipynb
from crawlers import crawlers


scope = ['https://www.googleapis.com/auth/spreadsheets']
creds = Credentials.from_service_account_file("C:\\Users\mayda\Downloads\\books-319701-17701ae5510b.json", scopes=scope)
gs = gspread.authorize(creds)
sheet = gs.open_by_url('https://docs.google.com/spreadsheets/d/17fJuHSGHnjHbyKJzTgzKpp1pe2J6sirK5QVjg2-8fFo/edit#gid=0')
worksheet = sheet.get_worksheet(0)

ISBN = "9789573317241"  # "哈利波特：神秘的魔法石"
# 設定 driver
my_options = Options()
my_options.add_argument("--incognito")  # 開啟無痕模式
# my_options.add_argument("--headless")  # 不開啟實體瀏覽器
capabilities = DesiredCapabilities.CHROME
capabilities["pageLoadStrategy"] = "none"

def organize_columns(df):    
    # 合併欄位
    # 處理 column 2：館藏地
    c2 = ["館藏地/室", "館藏室", "館藏地/館藏室", "館藏地", "典藏館"]
    df["c2"] = ""
    for c in c2:
        try:
            df["c2"] += df[c]
        except:
            pass
    
    # 處理 column 3：索書號
    c3 = ["索書號", "索書號/期刊合訂本卷期", "索書號 / 部冊號"]
    df["c3"] = ""
    for c in c3:
        try:
            df["c3"] += df[c]
        except:
            pass

    # 處理 column 4：館藏狀態
    c4 = ["狀態/到期日", "目前狀態 / 到期日", "館藏狀態", "處理狀態", '狀態 (說明)', '館藏現況 說明']
    df["c4"] = ""
    for c in c4:
        try:
            df["c4"] += df[c]
        except:
            pass
    
    # 解決編碼問題
    df["c2"] = df["c2"].str.replace("栅", "柵").str.replace("斈", "學")

    c1 = ["基隆市立公共圖書館", "國立臺灣海洋大學"]
    df["c1"] = ""
    c5 = [""]
    df["c5"] = ""

    # 直接生成另一個 DataFrame
    test = pd.DataFrame()
    test["圖書館"] = df["c1"]
    test["館藏地"] = df["c2"]
    test["索書號"] = df["c3"]
    test["館藏狀態"] = df["c4"]
    test["連結"] = df["c5"]

# perfect version!!!
# 國立宜蘭大學、佛光大學、嘉南藥理大學
def webpac_jsp_crawler(org, url, ISBN, driver, wait):
    try:
        # 進入＂搜尋主頁＂
        driver.get(url)

        # 定位＂下拉式選單＂，選擇 ISBN
        search_field = wait.until(
            EC.presence_of_element_located((By.NAME, "search_field"))
        )
        select = Select(search_field)
        select.select_by_value("ISBN")

        # 定位＂搜尋欄＂，輸入 ISBN
        search_input = wait.until(
            EC.presence_of_element_located((By.NAME, "search_input"))
        )
        search_input.send_keys(ISBN)
        search_input.submit()

        # 定位＂表格＂，爬取內容
        if wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.order"))):
            return pd.read_html(driver.page_source, encoding="utf-8")[-2]
    except:
        print(f"「{url}」無法爬取！")

# 宜蘭縣公共圖書館、桃園市立圖書館、高雄市立圖書館、屏東縣公共圖書館、花蓮縣圖書館、澎湖縣圖書館
def webpac_gov_crawler(org, url, ISBN, driver, wait):
    try:
        # 進入＂搜尋結果＂頁面
        search_url = url + ISBN
        driver.get(search_url)

        # 計算＂搜尋結果的數量＂
        book_list = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.booklist"))
        )
        results = len(book_list)
        print(f"搜尋結果數量有：{results}筆", end="    ")

        time.sleep(2)  # 頁面加載，用強制等待

        # 依照搜尋結果的數量分成：一筆和多筆兩種情形
        if results < 2:
            if wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.bookplace_list"))):
                click_more_btn(driver)
                return pd.read_html(driver.page_source, encoding="utf-8")[-1]
        else:
            soup = BeautifulSoup(driver.page_source, "html.parser")  # 用以當搜尋結果多筆時，找尋不同＂詳細書目＂的網址
            
            tgt = []
            url = url.split("search?")[0]  # 取得＂搜尋主頁＂頁面
            for li in soup.find_all("div", "booklist"):
                href = url + li.find("h2").find("a")["href"]
                
                driver.get(href)
                if wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.bookplace_list"))):
                    click_more_btn(driver)
                tgt.append(pd.read_html(driver.page_source, encoding="utf-8")[-1])
            return pd.concat(tgt, axis=0, ignore_index=True)
    except:
        print(f"「{url}」無法爬取！")

def Yilan(ISBN):
    output = []
    
    driver = webdriver.Chrome("C:\\Users\mayda\Downloads\chromedriver", options=my_options, desired_capabilities=capabilities)
    wait = WebDriverWait(driver, 10)
    
    output.append(
        crawlers.webpac_gov_crawler(
            "宜蘭縣公共圖書館", "https://webpac.ilccb.gov.tw/search?searchField=ISBN&searchInput=",
            ISBN,  driver, wait
        )
    )

    output.append(
        crawlers.webpac_jsp_crawler(
            "國立宜蘭大學", "https://lib.niu.edu.tw/webpacIndex.jsp",
            ISBN,  driver, wait
        )
    )

    output.append(
        crawlers.webpac_jsp_crawler(
            "佛光大學", "http://libils.fgu.edu.tw/webpacIndex.jsp",
            ISBN,  driver, wait
        )
    )
    
    driver.quit()
    return organize_columns(pd.concat(output, axis=0, ignore_index=True).fillna(""))

Yilan(ISBN)
aa = Yilan(ISBN)
worksheet.update([aa.columns.values.tolist()] + aa.values.tolist())

搜尋結果數量有：1筆    「https://webpac.ilccb.gov.tw/search?searchField=ISBN&searchInput=」無法爬取！
搜尋結果數量有：1筆    「https://webpac.ilccb.gov.tw/search?searchField=ISBN&searchInput=」無法爬取！


AttributeError: 'NoneType' object has no attribute 'columns'